# Определение текущего уровня потребительской лояльности

# Описание проекта

Необходимо определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Для оценки результатов опроса проставленные пользователями баллы разделены на три категории:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».
Значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). 

# Описание данных
Для проведения исследования данные выгрузили в SQLite — СУБД, в которой база данных представлена файлом. 

База данных содержит несколько таблиц.

<b> Таблица `user`содержит следующую информацию</b>:
* `user_id` — идентификатор клиента, первичный ключ таблицы;
* `lt_day` — количество дней «жизни» клиента;
* `age` — возраст клиента в годах;
* `gender_segment` — пол клиента (1 – женщина, 0 – мужчина);
* `os_name` — тип операционной системы;
* `cpe_type_name` — тип устройства;
* `location_id` — идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location;
* `age_gr_id` — идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment;
* `tr_gr_id` — идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment;
* `lt_gr_id` — идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment;
* `nps_score` — оценка клиента в NPS-опросе (от 1 до 10).

<b> Таблица `location`содержит следующую информацию</b>:
* `location_id` — идентификатор записи, первичный ключ;
* `country` — страна;
* `city` — город.

<b> Таблица `age_segment`содержит следующую информацию</b>:
* `age_gr_id` — идентификатор сегмента, первичный ключ;
* `bucket_min` — минимальная граница сегмента;
* `bucket_max` — максимальная граница сегмента;
* `title` — название сегмента.

<b> Таблица `traffic_segment`содержит следующую информацию</b>:
* `tr_gr_id` — идентификатор сегмента, первичный ключ;
* `bucket_min` — минимальная граница сегмента;
* `bucket_max` — максимальная граница сегмента;
* `title` — название сегмента.

<b> Таблица `lifetime_segment`содержит следующую информацию</b>:
* `lt_gr_id` — идентификатор сегмента, первичный ключ;
* `bucket_min` — минимальная граница сегмента;
* `bucket_max` — максимальная граница сегмента;
* `title` — название сегмента.

<a id="start"></a>
# Подключение к базе и выгрузка данных

С помощью кода ниже мы подключимся к базе данных, а также подготовим необходимую нам витрину через SQL-запрос и сохраним ее в csv-файл для дальнейшей работы в Tableau.

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

В рамках SQL-запроса мы:
- используем LEFT JOIN, т.к. ключевая таблица и данные которые должны быть сохранены находятся в таблице `user`;
- с помощью CASE мы задаем условные конструкции, чтобы выделить требуемый для анализа категории;
- с помощью SUBSTRING мы обрезаем первые три символа в названиях сегментов age, traffic, lifetime, т.к. названия имеют следующую структуру: "двузначный порядковый номер символ пробела название категории";
- кроме того, мы сразу удаляем пропуски из поля пол, их объем не превышает 1% и существенно не повлияет на результаты исследования.

In [3]:
query = """
SELECT user_id,
       lt_day,
       CASE 
            WHEN user.lt_day <= 365 THEN 'новый'
            WHEN user.lt_day > 365 THEN 'действующий'
       END AS is_new,
       age,
       CASE 
            WHEN user.gender_segment = 0 THEN 'мужчина'
            WHEN user.gender_segment = 1 THEN 'женщина'
       END AS gender_segment,
       os_name,
       cpe_type_name,
       l.country AS country,
       l.city AS city,
       SUBSTRING(a.title, 3) AS age_segment,
       SUBSTRING(tr.title, 3) AS traffic_segment,
       SUBSTRING(lt.title, 3) AS lifetime_segment,
       nps_score,
       CASE 
            WHEN user.nps_score <=6 THEN 'критики'
            WHEN user.nps_score BETWEEN 7 AND 8 THEN 'нейтралы'
            WHEN user.nps_score BETWEEN 9 AND 10 THEN 'сторонники'
       END AS nps_group
       
FROM user LEFT JOIN age_segment AS a ON user.age_gr_id = a.age_gr_id
LEFT JOIN location AS l ON user.location_id = l.location_id
LEFT JOIN traffic_segment AS tr ON user.tr_gr_id = tr.tr_gr_id
LEFT JOIN lifetime_segment AS lt ON user.lt_gr_id = lt.lt_gr_id
WHERE user.gender_segment IS NOT NULL
        
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,действующий,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,действующий,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,действующий,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

# Dashboard в Tableau public

<a href='https://public.tableau.com/app/profile/kiryl.panasik/viz/NPS_DB/sheet10'> Ссылка на дашборд «Общая информация о пользователях»</a>

<a href='https://public.tableau.com/app/profile/kiryl.panasik/viz/NPS_DB2/NPS_6'> Ссылка на дашборд «Анализ NPS и черты сторонников»</a>

<a id="conclusion"></a>
# Общие выводы

В результате анализа данных можно сделать следующие основные выводы:
- общее число участников опроса составило порядка 502 тыс. пользователей;
- основная масса пользователей - это лица возраста 25-54 лет, т.е. граждане платежеспособного возраста (на их долю приходится фактически 83% от общего числа опрошенных пользователей);
- доля действующих (со сроком жизни свыше 365 дн.) пользователей существенно выше новых пользователей (83% против 17%). 
При этом следует отметить, что наибольшее число новых пользователей приходится на категорию от 25 до 34 лет. В целом доля новых пользователей существенно выше в возрастных группах 25-44 года (31,6%-38,63%) и существенно превышает, как общую долю новых пользователей (17,03%), так и долю новых пользователей в остальных возрастных группах (там доля новых пользователей не превышает 14%).
- пользователи в целом являются лояльными по отношению к сервисам компании с уч. индекса NPS равного 21,97%;
- наибольшее число опрошенных проживают в крупных городах, таких как Москва, Санкт-Петербург, Новосибирск, Екатеринбург.
- а основная масса пользователей использует смартфон на андроид (порядка 85% пользователей) или смартфон на iOS (12% пользователей).

Что же касается динамики NPS следует отметить следующие ключевые моменты:
- женщины более лояльны к уровню сервиса нежели мужчины на протяжении всей жизни самого пользователя (т.е. в рамках всех возрастных групп);
- уровень лояльности растет с возрастом пользователей (это относится, как к мужчинам, так и женщинам);
- при этом с увеличение срока использования сервисов компании лояльность к сервису уменьшается;
- пользователи андроид существенно лояльнее пользователей iOS;
- а наиболее лояльные клиенты проживают в Москве и в целом западной части России (за искл. Воронежа с наихудшим уровнем лояльности в рамках выделенных категорий городов), наименее лояльные - в районе Урала и Сибири.

Оптимальным с т.зр. компании пользователем (сторонником сервиса) является пользователь со следующими характеристиками:
- женщина;
- срок использования сервисов до 1 года;
- место жительства - Москва;
- использует планшет или смартфон на андроиде;
- возраст - старший возрастной сегмент, в т.ч. пожилой (старше 50 лет).

Наименее лояльный пользователь в среднем:
- мужчина / парень;
- возраст до 25;
- пользуется iphone;
- срок использования сервисов - свыше 3 лет;
- проживает в Воронеже или в районе Урала и Сибири.

В целом, для повышения уровня лояльности мы можем рекомендовать компании уделить отдельное внимание работе с пользователями, которые давно являются клиентами сервиса, а также подростающему поколению, отдельно проанализировать причины столь низких показателей лояльности пользователей на iOS и уделить внимание сервису в ряде регионов РФ с низким уровнем лояльности.